# Face Recognition (Part 2: EigenFaces)

In this tutorial, we are going to use EigenFaces to recognize the faces of celebrities. We will use face images of a random subset of 10 celebrities, out of the 2,622.

Let's go through these step-by-step.

# Image Classification

In this experiment, we shall see:

- **1. FEATURE EXTRACTION**
    - extract eigenvalues and eigenvectors, and choose the best N principal components as features


- **2. TRAINING AND PREDICTION**:
    - We will train a Naive Bayes classifier on the extracted features and evaluate its performance


- **3. COMPARISON OF ALL ACCURACIES**: compare training, validation and testing accuracies.

Let us go through these step-by-step.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
DATA_ROOT = "/tmp/data/lab3"

# Recap of Part 1

In Part 1, we understood the data, and split it into train, val, and test. We then manipulated it so that the data is of uniform size, normalized, and mean subtracted. (Not that these operations can be performed on any data, not just images.)

We then saved the final datasets as "data.npz" file. Let us load them:

In [ ]:
# Loading data
data = np.load(DATA_ROOT+"/data.npz")

# Getting train, val and test data and labels, and the mean_image
data_train = data["data_train"]
labels_train = data["labels_train"]
data_val = data["data_val"]
labels_val = data["labels_val"]
data_test = data["data_test"]
labels_test = data["labels_test"]
mean_image = data["mean_image"]

# 1. FEATURE EXTRACTION (Eigen Faces)

## First $N$ Principal Components with maximum eigenvalues

As we have learnt in the lecture, PCA finds the set of orthonormal vectors which best
describe the distribution of the underlying dataset. In the given dataset, we have $n$
images of size $K \times K$. (We know that $K = 224$, and $n = 120$ in the training set)

Let us now see how the PCA features are extracted:

## 1.1 Find eigenvalues and eigenvectors

In [ ]:
def find_eigenvalues_and_eigenvectors(A):
    L = 1 / len(A.T) * np.dot(A, A.T)
    e, u = np.linalg.eig(L)
    w = e
    v = np.dot(A.T, u)
    return w, v

eigenvalues, eigenvectors = find_eigenvalues_and_eigenvectors(data_train)

## 1.2 Reordering, normalizing

- We need to reorder them so that they are in descending order of eigenvalues,
- Also, normalize the eigenvectors so that their norms are 1.

In [ ]:
# Find the required order of indices to make decreasing order of eigenvalue
sort_index = np.argsort(eigenvalues)[::-1]

# Use the calculated order of indices to reorder eigenvalues and eigenvectors
eigenvalues = eigenvalues[sort_index]
eigenvectors = eigenvectors[:, sort_index]

# NORMALIZE
eigenvectors = eigenvectors / np.linalg.norm(eigenvectors, axis=0)

## 1.3 Eigenfaces

The eigenvectors thus found are called eigenfaces (because we found the eigenvectors of faces...).

Since an eigenvector is of dimension ($K^2$,), it can be reshaped to $(K, K)$ and displayed as an image!

**Display the first 50 eigenfaces**

In [ ]:
# Number of eigenfaces to be plotted
N = 50
plt.figure(figsize=(10, 2*(N+5)//5))

for i in range(N):
    
    # Make a subplot
    plt.subplot((N + 5)//5, 5, i+1)
    
    # Plot the eigenface, after reshaping it to (224, 224)
    # Remember eigenfaces are **columns** in the matrix
    plt.imshow(np.reshape(eigenvectors[:, i], (224, 224)), cmap='gray')    
    # Turn off axis lines
    plt.axis("off")

plt.show()

## 1.4 Computing good value for $N$

In the given dataset, there are as many eigenvectors as the number of training examples. This can be verified by:

In [ ]:
eigenvectors.shape

Since each column is an eigenvector, there are 120 eigenvectors, each of 50176 dimensions. But usually, a smaller number $N$ of eigenvectors is chosen as a basis to make feature vectors. This forms the main point of PCA as we reduce the dimension of our representation while trying to capture the maximum variance in the data at the same time.

To decide the on the number $N$, i.e. the number of most important eigenvectors to keep as the basis, ((the cumulative sum of eigenvalues (assuming they are in decreasing order) divided by the total sum of eigenvalues)), vs. (( the number of eigenvalues considered ($M$) )) is plotted.

This plot basically tells us the fraction of total variance retained ($r$) vs. the number of eigenvalues considered ($M$) to retain that variance in data. This way, the plot gives a good understanding of the point of diminishing returns, i.e. the point where little variance is retained by retaining additional eigenvalues.

This can be understood by the following equation:

$$r = \frac{\sum_{k=1}^{M}\lambda_k}{\sum_{k=1}^{n}\lambda_k},\ \ \ \  M <= n$$

Plotting $r$ vs $M$ shall give a good idea of the impact of varying $M$ on $r$ and help you decide a good value of $N$ depending on how much variance (~80% is a popular choice) you want to retain.

As a part of **exercise**, we leave it to you to come to a conclusion for what will be a good value for N.
Let us choose $N = 20$ to continue with our experiment here.

In [ ]:
N = 20

This means we are choosing only $N$ **principal components**. In other words, we are choosing those $N (<< n(=120))$ eigenvectors that are most important in faces. We can look at the plots of the eigenfaces to see what sort of information we are choosing.

Let us note the first N principal components, i.e. the first N eigenvectors:

In [ ]:
pca_vectors = eigenvectors[:, :N]

## 1.5 Finding features using first $N$ Principal Components


Since we are using the most important eigenfaces as the _basis_ vectors, we need to project the data into these basis components to find the relevant features. We do this by finding the dot product of the data maxtrix and the matrix of the most important eigenvectors.

We know that the data (`data_train`, `data_val`, `data_test`) is of shape $n \times K^2$. We also know that the `pca_vectors` matrix is of shape $K^2 \times N$.

In [ ]:
pca_features_train = np.dot(data_train, pca_vectors)
pca_features_val = np.dot(data_val, pca_vectors)
pca_features_test = np.dot(data_test, pca_vectors)

Let's check the shapes of the features:

In [ ]:
print(pca_features_train.shape, pca_features_val.shape, pca_features_test.shape)

Hence, we can see that we have transformed our data from $n \times K^2$ to $n \times N$.

# 2. Training and Prediction

We will try a Naive Bayes Classifier for the multi class classification problem at hand. We have PCA features at our disposal.

## Naive Bayes algorithm
We have features and the labels present in the dataset. Now we need to learn naive bayes classifier for this dataset. We will go through a quick mathematical formulation of the Naive Bayes below.

The Naive Bayes Equation can be written as:

$$ P(Y = y|X) = P(X|Y = y)*P(Y = y) $$

where $X_i$ is a feature and $Y$ is the set of all labels. Therefore, we need to find the following 2 terms: $P(Y = y)$ called the **prior** and $P(X|Y = y)$ called the **likelihood**

### Calculate Prior $P(Y)$
This is also called the class probability. $P(Y)$ or $P(Y = y)$ is simply the fraction of the elements present in a class.

### Calculating Likelihood $P(X|Y = y)$ or $P(X_{i}|Y = y)$

For each feature $X_{i}$, we assume that they are uncorrelated and thus we can write,

$$ P(X|Y = y) = \Pi_{i} P(X_{i} | Y = y) $$

We wish to estimate $P(X_{i} | Y = y)$ for each $X_{i}$. Thus for samples corresponding to each class label $y$, we need to estimate a probability distribution. This is done by modelling the likelihood as a normal distribution (also called gaussian distribution),

$$ P(X_{i}|Y = y) = \frac{1}{\sqrt{2\pi v_{y, i}^{2}}} exp( - \frac{ (X_{i} - m_{y, i})^{2} }{2v_{y, i}^{2}} ) $$ 

As our features are real values, we estimate a normal distribution **corresponding to each feature** using all training samples per class.

Thus, we need to calculate mean ($ m_{y, i} $) and standard deviation ($ v_{y, i} $).

Once we've the mean and standard deviation for each feature and the prior probability for each class, we can use the above formulation to get the required probability $ P(X|Y = y) $ also called the **posterior probability**.
$$ P(X|Y = y) = (\Pi_{i}\frac{1}{\sqrt{2\pi v_{y, i}^{2}}} exp( - \frac{ (X_{i} - m_{y, i})^{2} }{2v_{y, i}^{2}} ))*P(Y = y) $$

We code the classifier using the functions below:

In [ ]:
## function to calculate prior for each class and the mean, std deviation for each feature
def get_prior_mean_dev(train_features, train_labels):

    classes, counts = np.unique(train_labels, return_counts=True)
    number_of_classes = len(classes)
    number_of_features = train_features.shape[1]

    # Prior
    prior = []
    for c, class_label in enumerate(classes):
        prior.append(counts[c]/len(train_labels))

    # Calculate the mean and variance per feature dimension here 
    # from the training set from samples belonging to each class label.
    means = np.zeros((number_of_features, number_of_classes)) # every feature, for each class
    std_dev = np.zeros((number_of_features, number_of_classes)) # every feature, for each class
    # For each class
    for c, class_label in enumerate(classes): # selecting a class 'y'
        class_rows = train_features[np.where(train_labels == class_label)[0], :]    # get all samples belonging to 'class_label'
        # For each feature
        for f in range(number_of_features):
            means[f, c] = np.mean(class_rows[:, f])
            std_dev[f, c] = np.std(class_rows[:, f])

    NB_classifier = {}
    NB_classifier['prior'] = prior
    NB_classifier['means'] = means
    NB_classifier['std_dev'] = std_dev

    return NB_classifier

## function to draw from the gaussian distribution for each feature 
def gaussian(x, m, v):
    return np.sqrt(1.0 / 2*np.pi) / v * np.exp(-0.5*(((x - m)/v)**2) )

## function to calculate likelihood based on features and their mean and deviation
def get_likelihood(features, means, std_dev):

    number_of_features, number_of_classes = means.shape

    # Feature probabilities
    # feature_probabilities = gaussian(tiled_features, means, std_dev) # get the probability
    feature_probabilities = np.zeros((number_of_features, number_of_classes))
    for f in range(number_of_features):
        for c in range(number_of_classes):
            feature_probabilities[f, c] = gaussian(features[f], means[f, c], std_dev[f, c])

    # Multiply all features' probabilities to get likelihood
    # Likelihood of each class
    likelihood = np.zeros((number_of_classes))
    for c in range(number_of_classes):
        likelihood[c] = np.prod(feature_probabilities[np.nonzero(feature_probabilities[:, c]), c]) # mutliply for each feature 'Xi'
    # likelihood = np.prod(feature_probabilities, axis=0)

    return likelihood

## function to get the classification accuracy based on Naive Bayes Classifier
def naive_bayes_classify(NB_classifier, test_features, test_labels):

    # NB_classifier is a dict with keys "prior", "means" and "std_dev"
    prior = NB_classifier["prior"]
    means = NB_classifier["means"]
    std_dev = NB_classifier["std_dev"]

    if len(test_features.shape) == 1:
        test_features = np.reshape(test_features, (1, len(test_features)))

    predicted_labels = []

    for t, test_sample_features in enumerate(test_features):
        # print("Progress: {0:0.04f}".format((t+1)/len(test_features)), end="\r")

        # Get the likelihood of the test samples belonging to each class
        likelihood = get_likelihood(test_sample_features, means, std_dev)

        # Calculate the approximate posterior = likelihood * prior
        approx_posterior = [np.asscalar(x*y) for x,y in zip(likelihood, prior)]
        #approx because of missing P(X) (constant) in the denominator

        # Make the prediction as that class with the maximum approximate posterior
        predicted_labels.append(np.argmax(approx_posterior))

    return np.mean(predicted_labels == test_labels)

In [ ]:
NB_clf = get_prior_mean_dev(pca_features_train, labels_train)
NB_pca_test_acc = naive_bayes_classify( NB_clf, pca_features_test, labels_test)
print(NB_pca_test_acc)

## Exercises
2. Find a good value for **N** (the number of eigenvectors to be used) by plotting $r$ vs $M$ curve and visualizing the number of eignevectors with a certain minimum variance (~80% is a popular choice) you want to retain.
3. Find both training and testing accuracy over the given dataset. Study the effect of changing train/test ratio (60% and 20% chosen initially) on the accuracy. Does the classifier overfit at large train:test ratio?
4. Instead of a Naive Bayes Classifier, can you train a small NN/CNN for this classification task? If yes, evaluate its performance.